## File Size

#### result
```
total 40K
-rwxrwxrwx 1 codespace codespace  16K Sep 29 19:44 Arithmetic
-rw-rw-rw- 1 codespace codespace  624 Sep 29 19:43 Arithmetic.c
-rw-rw-rw- 1 codespace codespace    0 Sep 29 20:31 Data.txt
-rw-rw-rw- 1 codespace codespace    0 Sep 29 20:30 Report_Lab_2.ipynb
-rwxrwxrwx 1 codespace codespace 8.7K Sep 29 19:50 Аrithmetic32
-rw-rw-rw- 1 codespace codespace 6.9K Sep 29 20:29 Аrithmetic32.asm

![](./Photo/Problem.png)

## Assemler code test

![](./Photo/Asm32Test1.png)
![](./Photo/Asm32Test2.png)
![](./Photo/Asm32Test3.png)
![](./Photo/Asm32Test4.png)
![](./Photo/Asm32Test5.png)

## C code test

![](./Photo/CTest1.png)
![](./Photo/CTest2.png)
![](./Photo/CTest3.png)
![](./Photo/CTest4.png)

## C code

```C
#include <stdio.h>
#include <stdlib.h>

int main(int argc, char *argv[]) {
    // check argument number
    if (argc != 4) {
        printf("Usage: %s a b c\n", argv[0]);
        printf("Where a, b, and c are integers\n");
        return 1;
    }
    
    // convert char to int
    int a = atoi(argv[1]);
    int b = atoi(argv[2]);
    int c = atoi(argv[3]);
    
    // check division by 0
    if (a == 0) {
        printf("Error: division by zero (a cannot be equal to 0)\n");
        return 1;
    }
    
    // (((b/a)+c)-a)
    int result = ((b / a) + c) - a;

    printf("Result: %d\n", result);
    
    return 0;
}

```

## Assembler code

```ASM
format ELF

public _start

macro syscall {
    int 0x80
}

macro cout _data, _length {
    push eax
    push ebx
    push edx
    push ecx
        mov eax, 4
        mov ebx, 1
        mov ecx, _data
        mov edx, _length
        syscall
    pop ecx
    pop edx
    pop ebx
    pop eax
}

macro cin _bufferInput, _length {
    push ebx
    push edx
    push ecx
        mov eax, 3
        mov ebx, 0
        mov ecx, _bufferInput
        mov edx, _length
        syscall
    pop ecx
    pop edx
    pop ebx
}

section '.data' writeable
    messageOptionProgram:
        db 'Choose one program:', 10
        db '1 - argument ASCII code', 10
        db '2 - arithmetic', 10, 10
        db 'q / Q - quite the programm', 10
    messageOptionEnd:
    kLengthMessageOptionProgram equ messageOptionEnd - messageOptionProgram

    messageInvalidNumberArgument:
        db 0x1B, '[H', 0x1B, '[J'
        db 'Error: write 1 or 3 additional arguments', 10, 10
    messageInvalidOptionEnd:
    kLengthMessageInvalidNumberArgument equ messageInvalidOptionEnd - messageInvalidNumberArgument

    messageInvalidDevider:
        db 0x1B, '[H', 0x1B, '[J'
        db 'Error: division by 0', 10, 10
    messageInvalidDeviderEnd:
    kLengthMessageInvalidDevider equ messageInvalidDeviderEnd - messageInvalidDevider

    messageResult:
        db 'Result:'
    messageResultEnd:
    kLengthMessageResult equ messageResultEnd - messageResult

    terminalClear db 0x1B, '[H', 0x1B, '[J'
    newLine db 10

    lengthInput dd 0
    lengthOutput dd 0

section '.bss' writeable
    bufferInput rb 256
    bufferOutput rb 256
    bufferCalculation rb 4

    operandA rb 4
    operandB rb 4
    operandC rb 4

section '.error' executable
    invalidNumberArgument:
        cout messageInvalidNumberArgument, kLengthMessageInvalidNumberArgument
        jmp return
    
    invalidDevider:
        cout messageInvalidDevider, kLengthMessageInvalidDevider
        jmp return

section '.function' executable
    ; input - address char eax
    ; output - int ecx
    CastCharInt:
        push ebx
        push edi
            ; check a minus
            xor ebx, ebx
            cmp byte [eax], '-'
            sete bl ; if equal bl = 1, else bl = 0

            ; skip a minus
            cmp bl, 1
            jne .startConvert
            inc eax

            .startConvert:
                xor ecx, ecx
            .cycleConvert:
                movzx edi, byte [eax]
                add ecx, edi
                sub ecx, '0'
                inc eax

                cmp byte [eax], 0
                je .cycleConvertEnd

                imul ecx, 10
                jmp .cycleConvert
            .cycleConvertEnd:

            cmp bl, 1
            jne .return

            imul ecx, -1

            .return:
        pop edi
        pop ebx
        ret
    
    ; input - int eax
    ; output - bufferOutput
    CastIntChar:
    push ecx
    push edi
    push edx
    push ebx
    push esi
        ; check a sign
        xor ebx, ebx
        cmp eax, 0
        jge .positive
        
        ; negative
        mov ebx, 1              ; negative flag
        neg eax                 ; make positive
    
        .positive:
            ; find number length
            push eax
            push ebx            ; save sign flag
                mov ecx, 0
                mov esi, 10
                
                test eax, eax   ; if zero number
                jnz .cycleIntLen
                mov ecx, 1      ; size 1 for zero number
                jmp .CycleDone
                
                .cycleIntLen:
                    xor edx, edx
                    div esi
                    inc ecx
                    test eax, eax
                    jnz .cycleIntLen
                .CycleDone:
            pop ebx             ; restore sign flag
            pop eax
            
            ; consider sign in size
            test ebx, ebx
            jz .unsign
            inc ecx
            
        .unsign:
            push ebx            ; save sign flag
                mov esi, 10
                mov edi, bufferOutput
                add edi, ecx
                mov byte [edi], 0
                dec edi
                
                .cycleConverter:
                    xor edx, edx
                    div esi
                    add dl, '0'
                    mov byte [edi], dl
                    dec edi
                    test eax, eax
                    jnz .cycleConverter
            .conversionDone:
            pop ebx
            
            ; add sign
            test ebx, ebx
            jz .return
            mov byte [edi], '-'
            
        .return:
            mov dword [lengthOutput], ecx
    pop esi
    pop ebx
    pop edx
    pop edi
    pop ecx
    ret

section '.text' executable    
_start:
    cout terminalClear, 6
main:
    pop eax
    cmp eax, 2
    je executionFirst

    cmp eax, 4
    je executionSecond

    jmp invalidNumberArgument

    ; get an argument's ASCII code
    executionFirst:
        cout terminalClear, 6

        pop eax ; get address of ./Arithmetic32

        pop eax ; get address of argument
        movzx eax, byte [eax] ; get the sumbol

        push eax
            mov ebx, 10
            xor ecx, ecx
            .cycleGetLength:
                xor edx, edx
                div ebx

                inc ecx

                cmp eax, 0
                jg .cycleGetLength
        pop eax

        mov edi, bufferOutput
        add edi, ecx
        dec edi
        xor esi, esi
        .cycleCastASCII:
            mov ebx, 10
            xor edx, edx
            div ebx
            add edx, '0'

            mov ebx, edx
            mov byte [edi], bl
            dec edi

            inc esi

            cmp eax, 0
            jne .cycleCastASCII
        
        mov dword [lengthOutput], esi

        cout messageResult, kLengthMessageResult
        cout newLine, 1
        cout bufferOutput, [lengthOutput]
        cout newLine, 1
        cout newLine, 1

        jmp return

    ; (((b/a)+c)-a)
    ; get the result of an arithmetic expression
    executionSecond:
        pop eax ; get address of ./Arithmetic32

        pop eax ; get address of the argument a
        call CastCharInt
        mov dword [operandA], ecx

        pop eax ; get address of the argument b
        call CastCharInt
        mov dword [operandB], ecx

        pop eax ; get address of the argument c
        call CastCharInt
        mov dword [operandC], ecx

        mov eax, dword [operandB]
        mov ebx, dword [operandA]
        test ebx, ebx
        jz invalidDevider
        cdq
        idiv ebx

        add eax, dword [operandC]

        sub eax, dword [operandA]

        call CastIntChar

        cout messageResult, kLengthMessageResult
        cout newLine, 1
        cout bufferOutput, [lengthOutput]
        cout newLine, 1
        cout newLine, 1

    return:
        mov eax, 1
        xor ebx, ebx
        syscall

```